In [ ]:
# Install Libraries

!pip install requests yfinance pandas


In [7]:
# Crypto Metadata Extraction
import requests, pandas as pd, time

all_records = []

for page in range(1, 6):
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=inr&per_page=250&order=market_cap_desc&page={page}&sparkline=False"
    response = requests.get(url)
    if response.status_code == 200:
        all_records.extend(response.json())
    time.sleep(1)

    coins_df = pd.DataFrame(all_records)

# Only keep required columns
coins_df = coins_df[[
    "id","symbol","name","current_price","market_cap",
    "market_cap_rank","total_volume","circulating_supply",
    "total_supply","ath","atl","last_updated"
]]


In [8]:
# Clean Crypto Metadata

records = []

for coin in all_records:
    records.append({
        "id": coin["id"],
        "symbol": coin["symbol"],
        "name": coin["name"],
        "current_price": coin["current_price"],
        "market_cap": coin["market_cap"],
        "market_cap_rank": coin["market_cap_rank"],
        "total_volume": coin["total_volume"],
        "circulating_supply": coin["circulating_supply"],
        "total_supply": coin["total_supply"],
        "ath": coin["ath"],
        "atl": coin["atl"],
        "last_updated": coin["last_updated"][:10]
    })

coins_df = pd.DataFrame(records)
coins_df.head()


,id,symbol,name,current_price,market_cap,market_cap_rank,total_volume,circulating_supply,total_supply,ath,atl,last_updated
0,bitcoin,btc,Bitcoin,6056570.00,121280444149268,1,1.020810e+13,1.998479e+07,1.998479e+07,11187013.00,3993.420000,2026-02-05
1,ethereum,eth,Ethereum,176121.00,21256329398206,2,5.293200e+12,1.206935e+08,1.206935e+08,431946.00,28.130000,2026-02-05
2,tether,usdt,Tether,90.09,16733082036111,3,1.599897e+13,1.857160e+11,1.911830e+11,105.52,36.860000,2026-02-05
3,binancecoin,bnb,BNB,59313.00,8103336849689,4,3.284140e+11,1.363598e+08,1.363598e+08,121422.00,2.580000,2026-02-05
4,ripple,xrp,XRP,116.53,7097927368064,5,7.325047e+11,6.091732e+10,9.998572e+10,313.99,0.159343,2026-02-05


In [9]:
# Historical Prices (Top 3 Coins)

top_coins = coins_df.sort_values("market_cap_rank").head(3)["id"]

price_frames = []

for coin_id in top_coins:
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=365"
    data = requests.get(url).json()

    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price_usd'])
    prices['date'] = pd.to_datetime(prices['timestamp'], unit='ms').dt.date
    prices['coin_id'] = coin_id

    price_frames.append(prices[['coin_id', 'date', 'price_usd']])
    time.sleep(1)

price_df = pd.concat(price_frames)
price_df.head()


,coin_id,date,price_usd
0,bitcoin,2025-02-06,96582.886829
1,bitcoin,2025-02-07,96635.418753
2,bitcoin,2025-02-08,96558.456310
3,bitcoin,2025-02-09,96558.237625
4,bitcoin,2025-02-10,96548.578654


In [10]:
# Oil Prices

oil_df = pd.read_csv("https://raw.githubusercontent.com/datasets/oil-prices/main/data/wti-daily.csv")
oil_df['Date'] = pd.to_datetime(oil_df['Date'])
oil_df = oil_df[(oil_df['Date'] >= '2020-01-01') & (oil_df['Date'] <= '2026-01-01')]
oil_df.rename(columns={'Date': 'date', 'Price': 'price_usd'}, inplace=True)
oil_df.head()


,date,price_usd
8569,2020-01-02,61.17
8570,2020-01-03,63.00
8571,2020-01-06,63.27
8572,2020-01-07,62.70
8573,2020-01-08,59.65


In [11]:
# Stock Data (Yahoo Finance API)

import yfinance as yf

tickers = ["^GSPC", "^IXIC", "^NSEI"]
stocks_df = yf.download(tickers, start="2020-01-01", end="2025-09-30", group_by="ticker")

stocks_df = stocks_df.stack(level=0).reset_index()
stocks_df.rename(columns={
    'level_1': 'ticker',
    'Date': 'date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

stocks_df.head()



[*********************100%***********************]  3 of 3 completed
C:\Users\relan\AppData\Local\Temp\ipykernel_25308\3081966752.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stocks_df = stocks_df.stack(level=0).reset_index()


Price,date,Ticker,open,high,low,close,volume
0,2020-01-01,^NSEI,12202.150391,12222.200195,12165.299805,12182.500000,3.041000e+05
1,2020-01-02,^GSPC,3244.669922,3258.139893,3235.530029,3257.850098,3.459930e+09
2,2020-01-02,^IXIC,9039.459961,9093.429688,9010.889648,9092.190430,2.862700e+09
3,2020-01-02,^NSEI,12198.549805,12289.900391,12195.250000,12282.200195,4.077000e+05
4,2020-01-03,^GSPC,3226.360107,3246.149902,3222.340088,3234.850098,3.484700e+09


In [26]:
# Create SQLite DB (LOCAL)

# import sqlite3

# conn = sqlite3.connect("cross_market.db")
# cursor = conn.cursor()

import sqlite3
import os

# Fix for Jupyter + Streamlit both
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

db_path = os.path.join(BASE_DIR, "cross_market.db")
conn = sqlite3.connect(db_path)


cursor = conn.cursor()



In [14]:
# Create Tables

cursor.execute("""CREATE TABLE IF NOT EXISTS cryptocurrencies(
    id TEXT PRIMARY KEY,
    symbol TEXT,
    name TEXT,
    current_price REAL,
    market_cap INTEGER,
    market_cap_rank INTEGER,
    total_volume INTEGER,
    circulating_supply REAL,
    total_supply REAL,
    ath REAL,
    atl REAL,
    last_updated DATE
);""")

cursor.execute("""CREATE TABLE IF NOT EXISTS crypto_prices(
    coin_id TEXT,
    date DATE,
    price_usd REAL
);""")

cursor.execute("""CREATE TABLE IF NOT EXISTS oil_prices(
    date DATE PRIMARY KEY,
    price_usd REAL
);""")

cursor.execute("""CREATE TABLE IF NOT EXISTS stock_prices(
    date DATE,
    ticker TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER
);""")

conn.commit()


In [21]:
# Insert Data into Tables

import pandas as pd

stock_df = pd.DataFrame({
    "date": pd.date_range("2025-01-01", periods=10),
    "ticker": ["^GSPC"] * 10,   # S&P 500
    "close": [4700,4720,4690,4715,4750,4780,4800,4790,4820,4850]
})



coins_df.to_sql("cryptocurrencies", conn, if_exists="append", index=False)
price_df.to_sql("crypto_prices", conn, if_exists="append", index=False)
oil_df.to_sql("oil_prices", conn, if_exists="append", index=False)
stock_df.to_sql("stock_prices", conn, if_exists="append", index=False)
conn.commit()




In [22]:
# Verify Tables

import pandas as pd
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,cryptocurrencies
1,crypto_prices
2,oil_prices
3,stock_prices
